In [1]:
import pandas as pd

df1 = pd.read_csv("/kaggle/input/d/shubhamprakash778/startup-investor-dataset/startup_investor_dataset (1).csv")
df2 = pd.read_csv("/kaggle/input/d/shubhamprakash778/startup-investor-dataset/startup_investor_dataset (2).csv")

df = pd.concat([df1, df2], axis=0)
df.to_csv("startup_investor_dataset.csv", index=False)
df.head(5)

,Details,Response
0,"Startup Details: Industry: Edtech, Stage: Beta...",- Score: 10 - Insight: The founder is in Edtec...
1,"Startup Details: Industry: DevOps, Stage: Alph...",- Score: 10 - Insight: The founder and investo...
2,Startup Details: Industry: Digital Therapeutic...,- Score: 15 - Insight: These two are a poor ma...
3,"Startup Details: Industry: SpaceTech, Stage: R...",Score: 15 Insight: The founder is in SpaceTech...
4,"Startup Details: Industry: Cloud Services, Sta...",- Score: 20 - Insight: This is a weak match. T...


In [2]:
M = df['Details'][0]
print(M)

0    Startup Details: Industry: Edtech, Stage: Beta...
0    Startup Details: Industry: VerticalFarming, St...
Name: Details, dtype: object


In [3]:
print(df1.shape)
print(df2.shape)
print(df.shape)

(5000, 2)
(3000, 2)
(8000, 2)


# Fine Tuning 

In [4]:
df = df[:4500]
df.shape

(4500, 2)

In [5]:
!pip install transformers datasets peft accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing i

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

In [7]:
def format_prompt(example):
    return {
        "text": f"### Instruction:\n{example['Details']}\n\n### Response:\n{example['Response']}"
    }

dataset = dataset.map(format_prompt)

Map:   0%|          | 0/4050 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [8]:
from huggingface_hub import login

login(token="")

In [9]:
# from transformers import AutoTokenizer

# # model_name = "google/gemma-2b-it"
# # tokenizer = AutoTokenizer.from_pretrained(model_name)

# model_name = "tiiuae/falcon-rw-1b"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def tokenize(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length", max_length=1024)

# tokenized_dataset = dataset.map(tokenize, batched=True)


from transformers import AutoTokenizer

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Map:   0%|          | 0/4050 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [10]:
# from transformers import AutoTokenizer

# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def tokenize(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# tokenized_dataset = dataset.map(tokenize, batched=True)


In [11]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.CAUSAL_LM,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(base_model, peft_config)


2025-05-18 04:56:42.094276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747544202.281841      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747544202.338249      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [12]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./falcon-rw-1b-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
   # evaluation_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps=2
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [13]:
import wandb

wandb.login(key="")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shubhamusingcolab9 (shubhamusingcolab9-iiit-manipur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250518_045721-4k4qzi6x
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./falcon-rw-1b-finetuned
wandb: ⭐️ View project at https://wandb.ai/shubhamusingcolab9-iiit-manipur/huggingface
wandb: 🚀 View run at https://wandb.ai/shubhamusingcolab9-iiit-manipur/huggingface/runs/4k4qzi6x
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is to

Step,Training Loss
500,1.231900
1000,0.842700
1500,0.781600
2000,0.754200
2500,0.738500
3000,0.722900
3500,0.714600
4000,0.707500
4500,0.701500
5000,0.698500


TrainOutput(global_step=5060, training_loss=0.788103917087962, metrics={'train_runtime': 42559.5147, 'train_samples_per_second': 0.952, 'train_steps_per_second': 0.119, 'total_flos': 3.005801734833439e+17, 'train_loss': 0.788103917087962, 'epoch': 9.981243830207305})

In [15]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# checkpoint_path = "/kaggle/working/gemma2b-finetuned/checkpoint-450"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
# model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
# model.to("cuda")

# input_text = "Startup Details: Industry: Edtech, Stage: Beta Testing, Funding Required: ₹250Cr+, Business Model: Equipment-as-a-Service, Traction: 50% MoM growth | Investor Details: Industry: Smart Packaging, Funding Stage: Strategic Investors, Investment Range: $50M+, Business Model: Profit-sharing, Geographical Preference: North-East India"
# inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
# outputs = model.generate(**inputs, max_new_tokens=200)
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# print("Output")
# print(response)
